![Podmanlogo](Pictures/podman-logo.png)

# Podman storage

As explained through the Podman 101 workshop, once a container is deleted all the data stored in it is also deleted. This is a problem for certain applications that may require to persist data.
Think of a containerized database, at some point you will update the database version, to achieve this you'll delete your old version container and deploy the new version container. In this example all the data stored in the database would be lost. Then, how can application data be preserved? Podman provides some different options for adding persistent storage and manage volumes, we are going to explore them throught this section of the workshop.

## Store Data on Host Machine

Podman provides mechanisms to use container host machine storage by using volumes and bind mounts. It provides the following benefits:
 - Data stored in the container host machine persists across container deletions.
 - As you're not using the Copy On Write (COW) filesystem within the container, it will usually provide better write performance.
 - Data can be shared between multiple containers. A typical examples is when one container is accessing to the volume for read/write operations while a second container is accessing only for reads.
 - It's possible to use external storage as you can host data mounts over the network, for examples by using NFS.

There are two ways of managing storage: volumes and binds. Volumes are fully managed by Podman while binds are data mounts managed by the user.

Let's begin with binds, first we need to create a directory in which we will store our data.

In [ ]:
!mkdir ~/my-persistent-storage

Now, lets move into the new directory and create an index.html file that we will use later for our containerized webserver.

In [ ]:
!cd ~/my-persistent-storage
!echo "This message is stored in the container host, in the directory ~/my-persistent-storage" > ~/my-persistent-storage/index.html
!cat index.html

Now we can use an nginx web server container image without any storage mounted to it.

In [ ]:
!podman run -it --rm -d -p 8080:80 --name my-web-server nginx

Verify it's running and showing the default message.

In [ ]:
!curl localhost:8080

We see the expected behaviour, it shows the default nginx web server file. During Podman 101 workshop we modified this message by creating a new custom container image. Now we are going to demonstrate how to use bind storage to achieve the same results.

In this case we will use the "-v" or "--volume" option in the command and then we will specify the directory to be mounted from the host and the directory in which to mount it inside the container. It has to be expressed in the "my-container-host-directory:my-container-mount-point:OPTIONS". The "OPTIONS" part is optional, but it's important to use the ":Z" argument if we have SElinux enabled in our system as it will the proper security context to our directory so our container can use the data within it.

Before we can test it we will stop the previous container:

In [ ]:
!podman stop my-web-server

Now we can create our new webserver with the bind storage:

In [ ]:
!podman run -it --rm -d -p 8080:80 --name my-web-server -v ~/my-persistent-storage:/usr/share/nginx/html:Z nginx

Check the message that it's showing:

In [ ]:
!curl localhost:8080

With this we have demonstrated how easy it is to use bind mounts and reutilize the same container image with different outputs just by modifying the mount. This is a way of improve efficiency and save storage space as you don't need to create a dedicated container image for each application, in this case for each web server.

Before we proceed with volume mounts lets remove our recently created container.

In [ ]:
!podman stop my-web-server

We've seen how to use binds, in that scenario the user was in responsible for creating a directory and mountin it to the container. With volumes it's Podman who manages the data mounts. You can manage volumes by using the podman volume command.

In [ ]:
!podman volume create my-persistent-volume

You can list all of your volumes

In [ ]:
!podman volume ls

And even get detailed information of them:

In [ ]:
!podman inspect my-persistent-volume

When you work with volumes in rootless containers, Podman stores the data in the $HOME/.local/share/containers/storage/volumes/ directory.

You can mount a volume using the same convention as we used with binds, but instead of specifying a local directory you just need to use the volume name:

In [ ]:
!podman run -it --rm -d --name my-persistent-data-container -v my-persistent-volume:/home fedora

As Podman is managing the volume you do not need to add the SElinux permissions.

As you can see, we have mounted our volume to the /home directory. Lets check whats inside it:

In [ ]:
!podman exec my-persistent-data-container ls /home

The directory is empty, just as expected. Now we will add a new file to it:

In [ ]:
!podman exec my-persistent-data-container touch /home/my-persistent-data

Now lets check again what's inside the /home directory:

In [ ]:
!podman exec my-persistent-data-container ls /home

As you can see our file is sitting there. Lets delete this container a create a new one without any volumes mounted to it.

First we delete all of our containers:

In [ ]:
!podman stop --all

Now we create a new one from the same image but without any volume mounted to it:

In [ ]:
!podman run -it --rm -d --name my-persistent-data-container fedora

Our data is not present any more:

In [ ]:
!podman exec my-persistent-data-container ls /home

But our volume is still there:

In [ ]:
!podman volume list

Hence, if we want to create a container with the same information we had before we just need to delete the newly created container and create a new one with our volume mounted to it:

In [ ]:
!podman stop --all
!podman run -it --rm -d --name my-persistent-data-container -v my-persistent-volume:/home fedora

If we check now the data inside the container we should find our file:

In [ ]:
!podman exec my-persistent-data-container ls /home

As you can see we can remount the data of a previous container in a new one. By doing this we simplify the process of managing storage in containers.

Remember you can delete all of your unused storage volumes by running the following command:

In [ ]:
!podman volume prune -f

Last but not least, you can export and import the data of Podman volumes by running the following commands:
 - podman volume import my_volume_name my_data.tar.gz
 - podman volume export my_volume_name --output my_data.tar.gz

These commands will help you migrate data between containers, volumes and even systems if combined with sftp or other file transfer protocols.